### ドライブのマウント

In [1]:
# Googleドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### ライブラリ・モジュールのインポート

In [2]:
# ライブラリの準備
!pip install timm
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from timm.scheduler import CosineLRScheduler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import copy
import random
import pickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.6 MB/s eta 0:00:00


### シード値の設定

In [3]:
# シード値を設定
def fix_seed(seed=1234):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

fix_seed(seed=1234)

### データセットの準備

In [4]:
# 平均値と標準偏差を計算するためのCIFAR10データセットの前処理を定義
pre_transform_cifar10 = transforms.Compose([
    transforms.ToTensor()
])

# 平均値と標準偏差を計算するためのCIFAR10データセットの読み込み
pre_train_dataset_cifar10 = datasets.CIFAR10(root='/content/data/', download=True, transform=pre_transform_cifar10)

# 平均値と標準偏差を計算するための変数を初期化
pre_mean_cifar10 = 0.0
pre_std_cifar10 = 0.0
pre_total_samples_cifar10 = len(pre_train_dataset_cifar10)

# データセットのすべてのデータポイントに対して平均値と標準偏差を計算
for data in pre_train_dataset_cifar10:
    pre_image, _ = data
    pre_mean_cifar10 += pre_image.mean(dim=(1, 2))  # テンソルのチャンネルごとに平均を計算
    pre_std_cifar10 += pre_image.std(dim=(1, 2))    # テンソルのチャンネルごとに標準偏差を計算

# データセット全体の平均値と標準偏差を計算
pre_mean_cifar10 /= pre_total_samples_cifar10
pre_std_cifar10 /= pre_total_samples_cifar10

print("データセット全体の平均値: ", pre_mean_cifar10)
print("データセット全体の標準偏差: ", pre_std_cifar10)

100%|██████████| 170498071/170498071 [00:18<00:00, 9003779.30it/s]


Extracting /content/data/cifar-10-python.tar.gz to /content/data/
データセット全体の平均値:  tensor([0.4914, 0.4822, 0.4465])
データセット全体の標準偏差:  tensor([0.2023, 0.1994, 0.2010])


In [5]:
# 学習用のCIFAR10データセットの前処理を定義
train_transform_cifar10 = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=pre_mean_cifar10, std=pre_std_cifar10)
    ])
# テスト用のCIFAR10データセットの前処理を定義
test_transform_cifar10 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=pre_mean_cifar10, std=pre_std_cifar10)
    ])

# 学習用のCIFAR10データセットの読み込み
train_dataset_cifar10 = torchvision.datasets.CIFAR10(root='/content/data/', train=True, transform=train_transform_cifar10, download=True)
# テスト用のCIFAR10データセットの読み込み
test_dataset_cifar10 = torchvision.datasets.CIFAR10(root='/content/data/', train=False, transform=test_transform_cifar10, download=True)

# 学習用のCIFAR10データローダーを作成
train_loader_cifar10 = torch.utils.data.DataLoader(dataset=train_dataset_cifar10, batch_size=512, shuffle=True, num_workers=2)
# テスト用のCIFAR10データローダーを作成
test_loader_cifar10 = torch.utils.data.DataLoader(dataset=test_dataset_cifar10, batch_size=512, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


## モデルの実装

In [6]:
# 1×1のサブネットワーク獲得用の畳み込みを定義
def supermaskconv1x1(in_channels, out_channels, stride=1):
    return SupermaskConv(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)

# 3×3のサブネットワーク獲得用の畳み込みを定義
def supermaskconv3x3(in_channels, out_channels, stride=1):
    return SupermaskConv(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

In [7]:
# サブネットワーク獲得用のバッチ正則化として、非アフィン正規化を使用する（学習可能なパラメータを使用しない）
class NonAffineBatchNorm(nn.BatchNorm2d):
    def __init__(self, dim):
        super(NonAffineBatchNorm, self).__init__(dim, affine=False)

In [8]:
# 各重みにスコアを付与したスコアをソートしてtop k%を使用することでサブネットワークを獲得する
class GetSubnet(autograd.Function):
    @staticmethod
    def forward(ctx, scores, k):
        # スコアを複製する
        out = scores.clone()
        # スコアを昇順でソートする
        _, idx = scores.flatten().sort()
        # top k%以下のスコアの数
        j = int((1 - k) * scores.numel())
        # flat_outとoutは同じメモリを参照する（flat_outを変更するとoutにも影響する）
        flat_out = out.flatten()
        # top k%の要素を1にする
        flat_out[idx[j:]] = 1
        # top k%以外の要素を0にする
        flat_out[idx[:j]] = 0
        return out

    @staticmethod
    def backward(ctx, g):
        # 逆伝播時に勾配gをそのまま伝える
        return g, None

In [9]:
# サブネットワーク獲得用の畳み込みを定義
class SupermaskConv(nn.Conv2d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # 重みと同じ形状のスコアを用意
        self.scores = nn.Parameter(torch.Tensor(self.weight.size()))
        # スコアを一様分布で初期化
        nn.init.kaiming_uniform_(self.scores, a=math.sqrt(5))
        # 重みの勾配を無効化する
        self.weight.requires_grad = False

    def _init_conv(self):
        # 重みを一様分布で初期化
        if self.init == 'kaiming_uniform':
            nn.init.kaiming_uniform_(self.weight, mode='fan_out', nonlinearity='relu')
        # 重みを正規分布で初期化
        elif self.init == 'kaiming_normal':
            nn.init.kaiming_normal_(self.weight, mode='fan_out', nonlinearity='relu')
        # 重みを符号つき定数で初期化
        elif self.init == 'signed_constant':
            fan = nn.init._calculate_correct_fan(self.weight, mode='fan_out')
            gain = nn.init.calculate_gain('relu')
            std = gain / math.sqrt(fan)
            self.weight.data = self.weight.data.sign() * std

    def set_init(self, init):
        # 初期化手法を設定
        self.init = init

    def set_prune_rate(self, prune_rate):
        # 刈り込み率を設定
        self.prune_rate = prune_rate

    @property
    def clamped_scores(self):
        # スコアとして非負の値を返すようにする（重要度を表す）
        return self.scores.abs()

    def forward(self, x):
        # サブネットワークを獲得
        subnet = GetSubnet.apply(self.clamped_scores, 1 - self.prune_rate)
        # サブネットワークでマスク
        w = self.weight * subnet
        x = F.conv2d(x, w, self.bias, self.stride, self.padding, self.dilation, self.groups)
        return x

In [10]:
# サブネットワーク獲得用のResidual Blocksを定義
class SupermaskBuildingBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.bn1 = NonAffineBatchNorm(in_channels)
        self.conv1 = supermaskconv3x3(in_channels, out_channels, stride)
        self.bn2 = NonAffineBatchNorm(out_channels)
        self.conv2 = supermaskconv3x3(out_channels, out_channels)
        self.dropout = nn.Dropout(p=0.3)
        self.relu = nn.ReLU(inplace=True)

        # 入力と出力のチャンネル数が異なる場合（strideが1より大きい場合）、ダウンサンプリング
        if in_channels != out_channels or stride > 1:
            self.shortcut = supermaskconv1x1(in_channels, out_channels, stride)
        else:
            self.shortcut = nn.Sequential()

    def forward(self, x):
        out = self.bn1(x)
        out = self.relu(out)
        out = self.conv1(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.conv2(out)
        # 残差写像と恒等写像の要素毎の和を計算
        out += self.shortcut(x)
        return out

In [11]:
class SupermaskWideResNet(nn.Module):
    def __init__(self, block, depth, k, num_classes=10):
        super().__init__()
        assert (depth - 4) % 6 == 0, "depth should be 6n + 4"
        n = (depth - 4) // 6
        channels = [16, 16 * k, 32 * k, 64 * k]
        self.conv1 = supermaskconv3x3(3, channels[0])
        # Residual Blocks（1)
        self.layer1 = self._make_layer(block, channels[0], channels[1], n)
        # Residual Blocks（2）
        self.layer2 = self._make_layer(block, channels[1], channels[2], n, stride=2)
        # Residual Blocks（3）
        self.layer3 = self._make_layer(block, channels[2], channels[3], n, stride=2)
        self.bn = NonAffineBatchNorm(channels[3])
        self.relu = nn.ReLU(inplace=True)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(channels[3], num_classes)

        # 重みの初期化
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # 初期化手法を設定
                m.set_init(init)
                # 刈り込み率を設定
                m.set_prune_rate(prune_rate)
                # 重みを初期化
                m._init_conv()

    # Residual Blocksを作成する関数を定義
    def _make_layer(self, block, in_channels, out_channels, blocks, stride=1):
        layers = []
        # 最初の Residual Block（stride=stride）
        layers.append(block(in_channels, out_channels, stride))
        # 残りの Residual Block（stride=1）
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

## 学習と評価（100 epochs, init=kaiming_uniform）

### prune_rate=0.999

In [153]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.999_uniform_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_from_CIFAR100_100epochs_pr0.999_uniform_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 29.314 %, Loss: 1.9843
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 43.624 %, Loss: 1.7559
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 50.242 %, Loss: 1.4006
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 53.31 %, Loss: 1.3620
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 54.088 %, Loss: 1.1950
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 54.658 %, Loss: 1.1987
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 55.234 %, Loss: 1.2516
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 55.184 %, Loss: 1.2165
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 55.59 %, Loss: 1.1932
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 56.44 %, Loss: 1.2763
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 56.274 %, Loss: 1.2122


In [160]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.999です


In [155]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [156]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [157]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 56.34 %


In [163]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.995

In [14]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.995_uniform_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_from_CIFAR100_100epochs_pr0.995_uniform_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 36.31 %, Loss: 1.8502
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 55.81 %, Loss: 1.5779
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 63.962 %, Loss: 0.9960
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 67.868 %, Loss: 0.8490
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 69.516 %, Loss: 0.9035
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 69.942 %, Loss: 0.8858
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 70.214 %, Loss: 0.7664
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 70.734 %, Loss: 0.9149
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 70.656 %, Loss: 0.8689
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 70.92 %, Loss: 0.8139
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 71.288 %, Loss: 0.7198


In [15]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.995です


In [16]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [17]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [18]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 75.97 %


In [19]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.99

In [20]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.99_uniform_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_from_CIFAR100_100epochs_pr0.99_uniform_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 35.55 %, Loss: 1.7883
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 59.102 %, Loss: 1.4889
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 68.766 %, Loss: 0.8502
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 72.258 %, Loss: 0.7280
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 73.554 %, Loss: 0.6751
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 74.314 %, Loss: 0.7467
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 75.062 %, Loss: 0.7801
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 74.996 %, Loss: 0.6922
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 75.5 %, Loss: 0.6909
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 76.082 %, Loss: 0.5890
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 76.264 %, Loss: 0.7350


In [21]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.990です


In [22]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [23]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [24]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 80.38 %


In [25]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.95

In [26]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.95_uniform_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_from_CIFAR100_100epochs_pr0.95_uniform_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 35.086 %, Loss: 1.8317
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 60.69 %, Loss: 1.5224
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 71.624 %, Loss: 0.6969
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 75.246 %, Loss: 0.7257
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 76.668 %, Loss: 0.6643
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 77.356 %, Loss: 0.5970
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 77.814 %, Loss: 0.6580
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 78.3 %, Loss: 0.5818
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 78.906 %, Loss: 0.5585
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 79.114 %, Loss: 0.5594
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 79.566 %, Loss: 0.5847


In [27]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.950です


In [28]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [29]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [30]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 82.34 %


In [31]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.9

In [12]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.9_uniform_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_from_CIFAR100_100epochs_pr0.9_uniform_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 35.9 %, Loss: 1.8280
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 62.598 %, Loss: 1.5529
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 72.126 %, Loss: 0.7235
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 75.44 %, Loss: 0.6376
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 77.21 %, Loss: 0.6060
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 77.812 %, Loss: 0.6146
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 78.378 %, Loss: 0.6004
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 78.748 %, Loss: 0.6591
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 79.09 %, Loss: 0.6867
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 79.528 %, Loss: 0.5814
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 79.502 %, Loss: 0.5193
Ep

In [13]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.900です


In [14]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [15]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [16]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 82.06 %


In [17]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.7

In [32]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.7_uniform_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_from_CIFAR100_100epochs_pr0.7_uniform_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 35.514 %, Loss: 1.8450
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 61.856 %, Loss: 1.5200
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 72.602 %, Loss: 0.7691
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 75.804 %, Loss: 0.6770
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 77.502 %, Loss: 0.5752
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 78.156 %, Loss: 0.5942
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 78.724 %, Loss: 0.4905
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 79.468 %, Loss: 0.5878
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 80.122 %, Loss: 0.5754
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 80.162 %, Loss: 0.5711
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 80.818 %, Loss: 0.53

In [33]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.700です


In [34]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [35]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [36]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 83.62 %


In [37]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.5

In [38]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.5_uniform_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_from_CIFAR100_100epochs_pr0.5_uniform_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 36.086 %, Loss: 1.8360
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 61.28 %, Loss: 1.4770
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 72.42 %, Loss: 0.7501
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 76.172 %, Loss: 0.7447
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 77.908 %, Loss: 0.6256
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 78.232 %, Loss: 0.7210
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 78.932 %, Loss: 0.6071
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 79.524 %, Loss: 0.5432
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 80.3 %, Loss: 0.6616
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 80.08 %, Loss: 0.5287
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 80.746 %, Loss: 0.5436
Ep

In [39]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.500です


In [40]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [41]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [42]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 83.61 %


In [43]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

## 学習と評価（100 epochs, init=kaiming_normal）

### prune_rate=0.999

In [44]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.999_normal_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_from_CIFAR100_100epochs_pr0.999_normal_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 30.01 %, Loss: 1.9564
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 43.122 %, Loss: 1.7583
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 49.56 %, Loss: 1.3881
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 52.748 %, Loss: 1.3419
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 53.85 %, Loss: 1.3152
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 54.28 %, Loss: 1.2419
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 53.814 %, Loss: 1.2585
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 55.402 %, Loss: 1.2870
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 55.302 %, Loss: 1.2216
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 55.494 %, Loss: 1.2213
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 55.738 %, Loss: 1.2015
E

In [45]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.999です


In [46]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [47]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [48]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 62.07 %


In [49]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.995

In [50]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.995_normal_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_from_CIFAR100_100epochs_pr0.995_normal_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 31.52 %, Loss: 1.8704
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 53.426 %, Loss: 1.6376
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 61.724 %, Loss: 1.0420
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 65.504 %, Loss: 0.9052
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 66.596 %, Loss: 0.8980
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 67.026 %, Loss: 0.9455
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 67.554 %, Loss: 0.7949
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 68.074 %, Loss: 0.8696
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 68.066 %, Loss: 0.9774
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 68.848 %, Loss: 0.8855
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 69.014 %, Loss: 0.840

In [51]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.995です


In [52]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [53]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [54]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 73.55 %


In [55]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.99

In [56]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.99_normal_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_from_CIFAR100_100epochs_pr0.99_normal_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 31.77 %, Loss: 1.8234
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 56.63 %, Loss: 1.5326
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 67.76 %, Loss: 0.8462
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 71.356 %, Loss: 0.7369
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 73.226 %, Loss: 0.7606
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 73.682 %, Loss: 0.6693
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 74.226 %, Loss: 0.7858
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 74.81 %, Loss: 0.7551
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 75.13 %, Loss: 0.7659
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 75.418 %, Loss: 0.7335
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 75.852 %, Loss: 0.6316
Ep

In [57]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.990です


In [58]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [59]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [60]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 79.24 %


In [61]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.95

In [62]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.95_normal_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_from_CIFAR100_100epochs_pr0.95_normal_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 38.012 %, Loss: 1.8183
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 61.604 %, Loss: 1.5160
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 71.734 %, Loss: 0.7719
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 75.432 %, Loss: 0.7367
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 76.662 %, Loss: 0.6276
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 77.438 %, Loss: 0.6622
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 78.078 %, Loss: 0.6308
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 78.246 %, Loss: 0.6515
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 79.228 %, Loss: 0.5579
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 79.034 %, Loss: 0.6360
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 79.43 %, Loss: 0.563

In [63]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.950です


In [64]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [65]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [66]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 82.68 %


In [67]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.9

In [68]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.9_normal_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_from_CIFAR100_100epochs_pr0.9_normal_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 34.484 %, Loss: 1.8429
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 60.924 %, Loss: 1.5423
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 71.644 %, Loss: 0.8207
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 75.164 %, Loss: 0.6483
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 76.66 %, Loss: 0.6767
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 77.288 %, Loss: 0.6264
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 77.98 %, Loss: 0.5983
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 78.648 %, Loss: 0.6811
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 78.806 %, Loss: 0.5824
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 79.188 %, Loss: 0.5539
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 79.598 %, Loss: 0.5227

In [69]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.900です


In [70]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [71]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [72]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 82.74 %


In [73]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.7

In [77]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.7_normal_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_from_CIFAR100_100epochs_pr0.7_normal_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 34.6 %, Loss: 1.8014
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 60.886 %, Loss: 1.5370
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 71.846 %, Loss: 0.8007
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 75.354 %, Loss: 0.6962
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 76.978 %, Loss: 0.6141
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 77.506 %, Loss: 0.6228
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 78.168 %, Loss: 0.6234
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 78.732 %, Loss: 0.5561
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 79.214 %, Loss: 0.4794
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 80.006 %, Loss: 0.6302
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 80.216 %, Loss: 0.5208

In [78]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.700です


In [79]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [80]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [81]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 82.79 %


In [82]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.5

In [83]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.5_normal_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_from_CIFAR100_100epochs_pr0.5_normal_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 36.25 %, Loss: 1.8549
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 61.88 %, Loss: 1.5166
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 72.696 %, Loss: 0.7224
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 76.044 %, Loss: 0.7009
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 77.692 %, Loss: 0.5624
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 78.316 %, Loss: 0.6667
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 78.812 %, Loss: 0.6082
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 79.4 %, Loss: 0.6347
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 79.764 %, Loss: 0.6184
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 80.182 %, Loss: 0.5062
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 80.518 %, Loss: 0.5349
E

In [84]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.500です


In [85]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [86]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [87]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 83.64 %


In [88]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

## 学習と評価（100 epochs, init=signed_constant）

### prune_rate=0.999

In [89]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.999_constant_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_from_CIFAR100_100epochs_pr0.999_constant_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 28.234 %, Loss: 1.9513
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 46.34 %, Loss: 1.7991
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 54.576 %, Loss: 1.2176
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 57.722 %, Loss: 1.1337
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 58.552 %, Loss: 1.2002
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 58.746 %, Loss: 1.3233
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 59.632 %, Loss: 1.2679
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 59.588 %, Loss: 1.2879
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 60.274 %, Loss: 1.2073
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 60.292 %, Loss: 1.0883
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 60.716 %, Loss: 1.166

In [90]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.999です


In [91]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [92]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [93]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 65.93 %


In [94]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.995

In [95]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.995_constant_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_from_CIFAR100_100epochs_pr0.995_constant_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 37.44 %, Loss: 1.8142
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 57.374 %, Loss: 1.4794
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 67.936 %, Loss: 0.8461
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 71.438 %, Loss: 0.7455
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 72.826 %, Loss: 0.8326
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 73.252 %, Loss: 0.8641
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 73.964 %, Loss: 0.7294
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 74.248 %, Loss: 0.7211
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 75.042 %, Loss: 0.6141
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 75.146 %, Loss: 0.7362
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 74.814 %, Loss: 0.717

In [96]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.995です


In [97]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [98]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [99]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 79.59 %


In [100]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.99

In [101]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.99_constant_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_from_CIFAR100_100epochs_pr0.99_constant_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 34.216 %, Loss: 1.8214
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 58.824 %, Loss: 1.4932
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 70.648 %, Loss: 0.7553
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 74.33 %, Loss: 0.7007
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 75.824 %, Loss: 0.6486
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 76.27 %, Loss: 0.7064
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 76.826 %, Loss: 0.5850
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 77.302 %, Loss: 0.7058
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 77.858 %, Loss: 0.5806
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 78.102 %, Loss: 0.6876
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 78.446 %, Loss: 0.6065

In [102]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.990です


In [103]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [104]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [105]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 81.68 %


In [106]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.95

In [107]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.95_constant_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_from_CIFAR100_100epochs_pr0.95_constant_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 34.73 %, Loss: 1.8311
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 61.886 %, Loss: 1.5059
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 72.21 %, Loss: 0.7394
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 75.734 %, Loss: 0.6446
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 77.02 %, Loss: 0.6888
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 77.922 %, Loss: 0.5995
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 78.486 %, Loss: 0.5539
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 78.748 %, Loss: 0.6760
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 78.916 %, Loss: 0.6588
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 79.722 %, Loss: 0.6034
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 80.21 %, Loss: 0.4943
E

In [108]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.950です


In [109]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [110]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [111]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 82.54 %


In [112]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.9

In [113]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.9_constant_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_from_CIFAR100_100epochs_pr0.9_constant_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 39.168 %, Loss: 1.8117
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 61.486 %, Loss: 1.4974
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 71.97 %, Loss: 0.8497
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 75.754 %, Loss: 0.5894
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 77.47 %, Loss: 0.6699
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 78.212 %, Loss: 0.5757
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 78.438 %, Loss: 0.5512
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 79.722 %, Loss: 0.6374
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 79.496 %, Loss: 0.6841
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 80.04 %, Loss: 0.5546
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 79.742 %, Loss: 0.5788


In [114]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.900です


In [115]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [116]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [117]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 83.18 %


In [118]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.7

In [119]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.7_constant_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_from_CIFAR100_100epochs_pr0.7_constant_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 34.808 %, Loss: 1.8381
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 63.188 %, Loss: 1.4911
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 72.692 %, Loss: 0.7605
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 76.382 %, Loss: 0.7291
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 77.706 %, Loss: 0.5324
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 78.65 %, Loss: 0.6275
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 79.058 %, Loss: 0.6477
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 79.678 %, Loss: 0.5719
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 80.176 %, Loss: 0.6087
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 80.064 %, Loss: 0.7597
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 80.538 %, Loss: 0.527

In [120]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.700です


In [121]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [122]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [123]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 82.99 %


In [124]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.5

In [125]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = torch.load('/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.5_constant_CLRS_restest.pth')
# 最終層を変更
model.fc = nn.Linear(model.fc.in_features, 10)
# 最終層以外のパラメータの勾配計算を停止
for param in model.parameters():
    param.requires_grad = False
# 最終層のパラメータのみを学習するように設定
for param in model.fc.parameters():
    param.requires_grad = True
# モデルをデバイスに転送
model = model.to(device)
# 変更後のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.01
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))


# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_from_CIFAR100_100epochs_pr0.5_constant_50epochs_lr0.01_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 36.238 %, Loss: 1.8601
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 61.986 %, Loss: 1.5224
Epoch [2/50], Learning Rate: 0.00208
Epoch [3/50], Train Accuracy: 72.812 %, Loss: 0.7060
Epoch [3/50], Learning Rate: 0.00406
Epoch [4/50], Train Accuracy: 76.15 %, Loss: 0.7351
Epoch [4/50], Learning Rate: 0.00604
Epoch [5/50], Train Accuracy: 77.25 %, Loss: 0.5531
Epoch [5/50], Learning Rate: 0.00802
Epoch [6/50], Train Accuracy: 78.488 %, Loss: 0.6231
Epoch [6/50], Learning Rate: 0.01
Epoch [7/50], Train Accuracy: 78.622 %, Loss: 0.6234
Epoch [7/50], Learning Rate: 0.009990232305719944
Epoch [8/50], Train Accuracy: 79.122 %, Loss: 0.5968
Epoch [8/50], Learning Rate: 0.009960967771506667
Epoch [9/50], Train Accuracy: 79.782 %, Loss: 0.5795
Epoch [9/50], Learning Rate: 0.00991232189110701
Epoch [10/50], Train Accuracy: 80.278 %, Loss: 0.4664
Epoch [10/50], Learning Rate: 0.009844486647586723
Epoch [11/50], Train Accuracy: 80.73 %, Loss: 0.4841


In [126]:
# モデルの刈り込み率を計算する関数を作成
def calculate_pruned_ratio(model):
    total_weights = 0
    total_pruned_weights = 0

    for module in model.modules():
        # モジュールがSupermaskConvであるか確認
        if isinstance(module, SupermaskConv):
            # SupermaskConv層の重みを取得
            weight = module.weight.data
            # 刈り込みを適用した後の重みを取得
            subnet = GetSubnet.apply(module.clamped_scores, 1 - module.prune_rate)
            pruned_weight = weight * subnet
            # 0である要素数を計算
            pruned_weights_count = (pruned_weight == 0).sum().item()
            total_pruned_weights += pruned_weights_count
            # 重みの全要素数を計算
            total_weights_count = pruned_weight.numel()
            total_weights += total_weights_count

    # 刈り込みが行われた割合を計算
    pruned_ratio = total_pruned_weights / total_weights
    return print(f"刈り込み率は{pruned_ratio:.3f}です")

# モデルの刈り込み率を確認
calculate_pruned_ratio(model)

刈り込み率は0.500です


In [127]:
# 学習前後のモデルのスコアを比較する関数を作成
def check_scores_change(model, model_init):
    # 各モデルの名前付きモジュールを順に調べる
    for (name, module), (name_init, module_init) in zip(model.named_modules(), model_init.named_modules()):
        # 両モジュールがConv2dのインスタンスであるかを確認
        if isinstance(module, torch.nn.Conv2d) and isinstance(module_init, torch.nn.Conv2d):
            # 両モジュールが'scores'属性を持っているかを確認
            if hasattr(module, 'scores') and hasattr(module_init, 'scores'):
                # 両モジュールの'scores'属性が一致しているかを確認
                # 一致していない場合、変更があったことを示すメッセージを出力
                if not torch.equal(module.scores, module_init.scores):
                    print(f'{name}のスコアが変化しています')
                    return
    # すべてのconv層で'scores'が変更されていない場合、その旨を出力
    print('すべてのスコアは変化していません')

# 学習前後でモデルのスコアが変化していないかを確認
check_scores_change(model, model_init)

すべてのスコアは変化していません


In [128]:
# 学習前後のモデルの最終層以外の重みを比較する関数を作成
def check_weight_change_except_last(model, model_init):
    for (name, param), (name_init, param_init) in zip(model.named_parameters(), model_init.named_parameters()):
        # 最終層を除く
        if name != 'fc.weight' and name != 'fc.bias':
            weights_changed = (param.data != param_init.data).any()
            if weights_changed:
                return print('最終層以外の重みが変化しています')
    return print('最終層以外の重みは変化していません')

# 学習前後でモデルの最終層以外の重みが変化していないかを確認
check_weight_change_except_last(model, model_init)

最終層以外の重みは変化していません


In [129]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 83.04 %


In [130]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()